# MHC Binding Prediction

In [1]:
import pandas as pd
import numpy as np
import numpy as np
import pandas as pd
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn import model_selection
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import roc_curve, precision_recall_curve
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn import metrics
import time

%matplotlib inline

Load experimental dataset

In [ ]:
train_csv = pd.read_csv("dataset/mhc_train.csv")
print(train_csv)

columns = ['mhc', 'seq_1', 'seq_2', 'seq_3', 'seq_4',
       'seq_5', 'seq_6', 'seq_7', 'seq_8', 'seq_9']

train_csv['seq_1'], train_csv['seq_2'], train_csv['seq_3'], train_csv['seq_4'], train_csv['seq_5'], train_csv['seq_6'], train_csv['seq_7'], train_csv['seq_8'], train_csv['seq_9'] = list(train_csv['sequence'].str)

def show_matrix(m):
    cm = sns.light_palette("seagreen", as_cmap=True)
    display(m.style.background_gradient(cmap=cm))

`Encode peptides` 
\Carolyn: please investigate previous work on encoding method, including not limited to,
https://dmnfarrell.github.io/bioinformatics/mhclearning
* one-hot
* sliding window (3)
* NLF
* Blosum


`One-hot`

In [ ]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore')

for col in columns:
    train_csv[col] = enc.fit_transform(train_csv[col])

y = train_csv['meas']
X = train_csv.drop(['pep_class', 'sequence'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0, stratify=y)
class_train = X_train['pep_class']
X_train.drop('pep_class', axis=1)
class_test = X_test['pep_class']
X_test.drop('pep_class', axis=1)

print(X_train)

`Metrics/Evaluation with AUC`  

In [ ]:
def auc_score(true,sc,cutoff=None):
    if cutoff!=None:
        true = (true<=cutoff).astype(int)
        sc = (sc<=cutoff).astype(int)        
    fpr, tpr, thresholds = metrics.roc_curve(true, sc, pos_label=1)
    r = metrics.auc(fpr, tpr)
    return  r

`Model 1: MLP regression`

In [ ]:
from sklearn.neural_network import MLPRegressor

# start with amino acid + 1
model = MLPRegressor(hidden_layer_sizes=(10), alpha=0.01, max_iter=500, activation='identity', solver='adam', random_state=2)
model.fit(X_train, y_train)
y_predict = model.predict(X_test)
class_predict = (y_predict > 0.5)

plt.figure(figsize=(10,10))
plt.scatter(class_test, class_predict, c='crimson')
plt.axis('equal')
plt.show()